# Conversation between DialogueGpt and Leolani

In this notebook, we create a loop in which DialogGpt has a conversation with Leolani. We send Leolani's initial prompt to DialogueGpt to start a conversation.
Next we capture Leolani's response from the brain and DialogueGpt's response constinuously until we meet the stop condition. Extracted triples are posted to a brain called DialueGpt.

In principle, this conversation can go on forever. At the end, we save the scenario in EMISSOR.

Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Import a conversation agent pipeline

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [18]:
from transformers import pipeline, Conversation
conversational_pipeline = pipeline("conversational", model='microsoft/DialoGPT-medium')
## DialogGPT at work

## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [7]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "DialogGpt"
HUMAN_ID = "DialogGpt"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-11-29-22:21:50 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [8]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/dialogGpt",
                                log_dir=log_path,
                                clear_all=True)

2021-11-29 22:22:05,852 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-11-29 22:22:12,200 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-29 22:22:13,563 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-11-29 22:22:13,565 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-11-29 22:22:13,567 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-11-29 22:22:13,568 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-11-29 22:22:13,675 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [9]:
replier = LenkaReplier()

2021-11-29 22:22:13,680 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [10]:
chat = Chat(HUMAN_ID)

2021-11-29 22:22:16,001 -     INFO - cltl.triple_extraction.api.Chat (DialogGpt)         000 - << Start of Chat with DialogGpt >>


## Start the interaction

In [11]:
print_details=False

t1 = datetime.now()

#### Initial prompt by the system from which we create a TextSignal and store it
leolani_prompt = f"{choice(TALK_TO_ME)}"
print('\n\t'+AGENT + ": " + leolani_prompt)
textSignal = d_util.create_text_signal(scenario_ctrl, leolani_prompt)
scenario_ctrl.append_signal(textSignal)
conv = Conversation()
conv.add_user_input(leolani_prompt)

utterance = ""
#### Get input and loop
while (datetime.now()-t1).seconds <= 120:
    ###### Getting the next input signals
    conversational_pipeline(conv)
    utterance = conv.generated_responses[-1]
    if not utterance:
        leolani_prompt = f"{choice(TALK_TO_ME)}"
        print('\n\t'+AGENT + ": " + leolani_prompt)
        textSignal = d_util.create_text_signal(scenario_ctrl, leolani_prompt)
        scenario_ctrl.append_signal(textSignal)
        conv.add_user_input(leolani_prompt)
    else:
        print('\n\t'+HUMAN_NAME + ": " + utterance)
        textSignal = d_util.create_text_signal(scenario_ctrl, utterance)
        scenario_ctrl.append_signal(textSignal)

        #### Process input and generate reply

        capsule, leolani_prompt = talk.process_text_and_reply(scenario_ctrl,
                               place_id,
                               location,
                               HUMAN_ID,
                               textSignal,
                               chat,
                               replier,
                               my_brain,
                               print_details)

        print('\n\t'+AGENT + ": " + leolani_prompt)
        textSignal = d_util.create_text_signal(scenario_ctrl, leolani_prompt)
        scenario_ctrl.append_signal(textSignal)
        conv.add_user_input(leolani_prompt)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



	Leolani2: Tell me anything, I love learning things

	DialogGpt: I'm a student at the University of Texas at Austin.
2021-11-29 22:22:19,308 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-11-29 22:22:19,328 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-11-29 22:22:19,332 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-11-29 22:22:24,302 -     INFO - cltl.triple_extraction.api.Chat (DialogGpt)         001 -  DialogGpt: "I'm a student at the University of Texas at Austin."
2021-11-29 22:22:28,843 -  WARNING -      cltl.brain.basic_brain.TypeReasoner - Failed to query Wikidata: HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=3)
2021-11-29 22:22:29,576 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of student-at-the-University-of-Texas-at-Austin. to: None
2021-11-29 22:22:29,624 -     INFO -    cltl.brain.basic_brain.LongTermMemory -

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



	Leolani2: Interesting! I did not know anything that dialoggpt be

	DialogGpt: I'm a student at the University of Texas at Austin.
2021-11-29 22:22:37,183 -     INFO - cltl.triple_extraction.api.Chat (DialogGpt)         002 -  DialogGpt: "I'm a student at the University of Texas at Austin."
2021-11-29 22:22:40,264 -  WARNING -      cltl.brain.basic_brain.TypeReasoner - Failed to query Wikidata: HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=3)
2021-11-29 22:22:41,122 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Reasoned type of student-at-the-University-of-Texas-at-Austin. to: None
2021-11-29 22:22:41,142 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Triple in statement: dialoggpt_be_a-student-at-the-university-of-texas-at-austin [agent_->_])
2021-11-29 22:22:41,148 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Statement Novelty: 1 times, e.g. dialoggpt on November,2021
2021-11-29 22:22:42,557 -     INFO -  cltl.b

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


	Leolani2: Do you have any gossip?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



	Leolani2: Do you have any gossip?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



	Leolani2: Would you like to chat? I'll do my best to keep up

	Leolani2: Tell me something, anything!


In [14]:
conv.generated_responses

["I'm a student at the University of Texas at Austin.",
 "I'm a student at the University of Texas at Austin.",
 "I don't know. I'm not a student at the University of Texas at Austin.",
 "I'm a student at the University of Texas at Austin.",
 '',
 'I love you',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

## Save the scenario data

In [15]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook